In [ ]:
import sys
sys.path.append("/bigdata/hplsim/external/software/picongpu/workshop/lib/python/")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import h5py as h5
import scipy.constants as const
from matplotlib.colors import LogNorm

from picongpu.plugins.data import radiation



In [ ]:
path_to_sim = "<pat_to_your_sim>"

# Classical radiation:


## High Harmonics Generation (HHG) 


In [ ]:
# load radiation text data

# select time step
ts = 5000
# load data
rad_data = np.loadtxt(path_to_sim+"/simOutput/<...>".format(ts))

In [ ]:
# manually define frequency axis

omega_rad = np.linspace(??, ??, 128) # in units of the laser frequency


In [ ]:
#plot spectrum:
plt.figure()
plt.plot(omega_rad, rad_data)

# add lines for harmonics:
plt.vlines(np.arange(10)+1, ymin=np.amin(rad_data), ymax=np.amax(rad_data), alpha=0.5)

plt.yscale('log')

plt.xlabel(r"$\omega_\mathrm{rad} \, \mathrm{[\omega_0]}$", fontsize=18)
plt.xticks(fontsize=14)

plt.ylabel(r"$\frac{\mathrm{d}^2I}{\mathrm{d}\omega \mathrm{d} \Omega} \, \mathrm{[Js]}$", fontsize=18)
plt.yticks(fontsize=14)

plt.show()

In [ ]:
# plot radiation over time:

# radiation dump steps (100, 200, 300, ..., 4900, 5000)
rad_dump_timesteps = np.arange(100, 5000+1, 100)

# create 2D array for all spectra
spectrum_over_time = np.empty((len(rad_dump_timesteps), len(omega_rad)))

# load spectra for each dump 
for i, ts in enumerate(rad_dump_timesteps):
    spectrum_over_time[i, :] = np.loadtxt(path_to_sim+"/simOutput/totalRad/e_radiation_{}.dat".format(ts))


In [ ]:
#plot radiation over time

# define the PIC time step
delta_t = 9.20429e-18 # [s]


plt.figure()
plt.pcolormesh(omega_rad, # x-axis
               rad_dump_timesteps * delta_t / 1e-15, # y-axis 
               spectrum_over_time, # 2D data set
               norm=LogNorm(), vmin=1e-28)

cb = plt.colorbar()

plt.xlabel(r"$\omega_\mathrm{rad} \, \mathrm{[\omega_0]}$", fontsize=18)
plt.xticks(fontsize=14)

plt.ylabel(r"$t \, \mathrm{[fs]}$", fontsize=18)
plt.yticks(fontsize=14)

cb.set_label(r"$\frac{\mathrm{d}^2I}{\mathrm{d}\omega \mathrm{d} \Omega} \, \mathrm{[Js]}$", fontsize=18)
for i in cb.ax.get_yticklabels():
    i.set_fontsize(14)

plt.show()

## Reading data with the radiation python module

In [ ]:
# select time step
ts = 5000
# load data with radiation module
rad_hdf5_filename = path_to_sim+"/simOutput/radiationHDF5/e_radAmplitudes_{}_0_0_0.h5".format(ts)
rad_data_viaModule = radiation.RadiationData(rad_hdf5_filename)

In [ ]:
# try to see what is in this class object by pressing tab tab
rad_data_viaModule.

### Work in progress:

**Ohhh no!!!**
There is not yet an **easy to use method** to get the frequencies!
See [issue #2759](https://github.com/ComputationalRadiationPhysics/picongpu/issues/2759)

But they are in the hdf5 data - so let us try to extract them later.

In [ ]:
#plot spectrum:
plt.figure()
plt.plot(omega_rad, rad_data_viaModule.get_Spectra()[0, :], label="all", lw=2)
plt.plot(omega_rad, rad_data_viaModule.get_Polarization_X()[0, :], "--", label="x-pol")
plt.plot(omega_rad, rad_data_viaModule.get_Polarization_Z()[0, :], "--", label="z-pol")

# add lines for harmonics:
plt.vlines(np.arange(10)+1, ymin=1e-32, ymax=1e-22, alpha=0.5)

plt.yscale('log')

plt.xlabel(r"$\omega_\mathrm{rad} \, \mathrm{[\omega_0]}$", fontsize=18)
plt.xticks(fontsize=14)

plt.ylabel(r"$\frac{\mathrm{d}^2I}{\mathrm{d}\omega \mathrm{d} \Omega} \, \mathrm{[Js]}$", fontsize=18)
plt.yticks(fontsize=14)

plt.legend(loc=4, fontsize=14)

plt.show()

... so now lets look for the frequency axis

In [ ]:
# access the hdf5 file via the module
list(rad_data_viaModule.h5_file['/data/5000/DetectorMesh/DetectorFrequency/'].items())

In [ ]:
omega_rad_viaModule = (rad_data_viaModule.h5_file['/data/5000/DetectorMesh/DetectorFrequency/omega'][0, :, 0]
                       * rad_data_viaModule.h5_file['/data/5000/DetectorMesh/DetectorFrequency/omega'].attrs['unitSI']
                       )

In [ ]:
lambda_laser = 800.0e-9 # [m]
omega_laser = 2.0 * np.pi * const.speed_of_light / lambda_laser

plt.plot(omega_rad_viaModule/omega_laser, rad_data_viaModule.get_Polarization_X()[0, :])

plt.xlabel(r"$\omega_\mathrm{rad} \, \mathrm{[\omega_0]}$", fontsize=18)
plt.xticks(fontsize=14)

plt.ylabel(r"$\frac{\mathrm{d}^2I}{\mathrm{d}\omega \mathrm{d} \Omega} \, \mathrm{[Js]}$", fontsize=18)
plt.yticks(fontsize=14)

plt.ylim(0, 3e-24)

plt.show()

# Bremsstrahlung:
## Reading the raw calorimeter data:

There is currently no module for reading calorimeter data via a python module.
You need to get your hands dirty by **reading openPMD flavored hdf5 data directly**.
See [issue #2760](https://github.com/ComputationalRadiationPhysics/picongpu/issues/2760).

However, everything you need can be found in the documentation:
https://picongpu.readthedocs.io/en/0.4.0/usage/plugins/particleCalorimeter.html

In [ ]:
# select time step
ts = 5000
# load calorimeter data with h5py
f_brems = h5.File(path_to_sim+"/simOutput/ph_b_calorimeter/all/ph_b_calorimeter_bremsstrahlung_all_{}_0_0_0.h5".format(ts))


In [ ]:
print(list(f_brems['/data/{}/'.format(ts)].items()))

In [ ]:
cal_brems_h = f_brems['/data/{}/calorimeter'.format(ts)]

cal_brems_data = cal_brems_h[...] * cal_brems_h.attrs['unitSI']

In [ ]:
N_energy, N_yaw, N_pitch = np.shape(cal_brems_data)
print(N_energy, N_yaw, N_pitch)

In [ ]:
for i in cal_brems_h.attrs.items():
    print(i)

In [ ]:
yaw = np.linspace(cal_brems_h.attrs['posYaw[deg]'] - cal_brems_h.attrs['maxYaw[deg]'],
                  cal_brems_h.attrs['posYaw[deg]'] + cal_brems_h.attrs['maxYaw[deg]'],
                  N_yaw)

pitch = np.linspace(cal_brems_h.attrs['posPitch[deg]'] - cal_brems_h.attrs['maxPitch[deg]'],
                    cal_brems_h.attrs['posPitch[deg]'] + cal_brems_h.attrs['maxPitch[deg]'],
                    N_pitch)

energy = np.linspace(cal_brems_h.attrs['minEnergy[keV]'], 
                     cal_brems_h.attrs['maxEnergy[keV]'], 
                     N_energy)

In [ ]:
# spatial energy distribution
# sum up the energy spectrum
plt.pcolormesh(yaw, 
               pitch, 
               np.sum(cal_brems_data, axis=0))
plt.xlabel(r"$\phi\ \mathrm{[^\circ]}$", fontsize=18)
plt.xticks(fontsize=14)
plt.ylabel(r"$\theta\ \mathrm{[^\circ]}$", fontsize=18)
plt.yticks(fontsize=14)
plt.show()

# energy spectrum
# sum up all solid angles
plt.plot(energy / 1e3,
         np.sum(cal_brems_data, axis=(1,2)))
plt.xlabel(r"$E_\mathrm{photons}\ \mathrm{[MeV]}$", fontsize=18)
plt.xticks(fontsize=14)
plt.ylabel(r"$E_\mathrm{cal} \mathrm{[\frac{J}{bin}]}$", fontsize=18)
plt.yticks(fontsize=14)
plt.xlim(0,10)
plt.show()

In [ ]:
f_brems.close()

# Compton scattering:
## Reading the raw calorimeter data:

There is currently no module for reading calorimeter data via a python module.
You need to get your hands dirty by **reading openPMD flavored hdf5 data directly**.
See [issue #2760](https://github.com/ComputationalRadiationPhysics/picongpu/issues/2760).

However, everything you need can be found in the documentation:
https://picongpu.readthedocs.io/en/0.4.0/usage/plugins/particleCalorimeter.html

**The same procedure as every year, James.**

In [ ]:
# select time step
ts = 5000
# load calorimeter data with h5py
f_compton = h5.File(path_to_sim+"/simOutput/ph_s_calorimeter/all/ph_s_calorimeter_compton_all_{}_0_0_0.h5".format(ts))

cal_compton_h = f_compton['/data/{}/calorimeter'.format(ts)]
cal_compton_data = cal_compton_h[...] * cal_compton_h.attrs['unitSI']
N_energy, N_yaw, N_pitch = np.shape(cal_compton_data)
yaw = np.linspace(cal_compton_h.attrs['posYaw[deg]'] - cal_compton_h.attrs['maxYaw[deg]'],
                  cal_compton_h.attrs['posYaw[deg]'] + cal_compton_h.attrs['maxYaw[deg]'],
                  N_yaw)

pitch = np.linspace(cal_compton_h.attrs['posPitch[deg]'] - cal_compton_h.attrs['maxPitch[deg]'],
                    cal_compton_h.attrs['posPitch[deg]'] + cal_compton_h.attrs['maxPitch[deg]'],
                    N_pitch)

energy = np.linspace(cal_compton_h.attrs['minEnergy[keV]'], 
                     cal_compton_h.attrs['maxEnergy[keV]'], 
                     N_energy)

# spatial energy distribution
# sum up the energy spectrum
plt.pcolormesh(yaw, 
               pitch, 
               np.sum(cal_compton_data, axis=0))
plt.xlabel(r"$\phi\ \mathrm{[^\circ]}$", fontsize=18)
plt.xticks(fontsize=14)
plt.ylabel(r"$\theta\ \mathrm{[^\circ]}$", fontsize=18)
plt.yticks(fontsize=14)
plt.show()

# energy spectrum
# sum up all solid angles
plt.plot(energy,
         np.sum(cal_compton_data, axis=(1,2)))
plt.xlabel(r"$E_\mathrm{photons}\ \mathrm{[keV]}$", fontsize=18)
plt.xticks(fontsize=14)
plt.ylabel(r"$E_\mathrm{cal} \mathrm{[\frac{J}{bin}]}$", fontsize=18)
plt.yticks(fontsize=14)
plt.yscale('log')
plt.xlim(0, 100)
plt.show()

f_compton.close()

# Probe Particles

still some open issues:
see [issue #2763](https://github.com/ComputationalRadiationPhysics/picongpu/issues/2763)

Documentation: https://picongpu.readthedocs.io/en/0.4.0/usage/workflows/probeParticles.html

In [ ]:
# select time step
ts = 2500
# load calorimeter data with h5py
f_probe = h5.File(path_to_sim+"/simOutput/h5/simData_{}.h5".format(ts), "r")

print(list(f_probe['/data/{}/particles/probe'.format(ts)].items()))

probe_h = f_probe['/data/{}/particles/probe'.format(ts)]

pos_probe_y = probe_h["positionOffset/y"][:] + probe_h["position/y"][:]
pos_probe_Ex = probe_h['probeE/x'][:] * probe_h['probeE/x'].attrs['unitSI']


f_probe.close()

In [ ]:
plt.plot(pos_probe_y, pos_probe_Ex)

plt.xlabel("y [cells]", fontsize=18)
plt.xticks(fontsize=14)

plt.ylabel(r"$E_x \, \mathrm{[V/m]}$", fontsize=18)
plt.yticks(fontsize=14)

plt.show()

In [ ]:
plt.hist2d(pos_probe_y, pos_probe_Ex, bins=(512, 256))

plt.xlabel("y [cells]", fontsize=18)
plt.xticks(fontsize=14)

plt.ylabel(r"$E_x \, \mathrm{[V/m]}$", fontsize=18)
plt.yticks(fontsize=14)
plt.show()

In [ ]:
sorted_output = np.array(sorted(zip(pos_probe_y,pos_probe_Ex)))
print(np.shape(sorted_output))

plt.plot(sorted_output[:, 0], sorted_output[:, 1])

plt.xlabel("y [cells]", fontsize=18)
plt.xticks(fontsize=14)

plt.ylabel(r"$E_x \, \mathrm{[V/m]}$", fontsize=18)
plt.yticks(fontsize=14)

plt.show()

# Tracer Particles

Documentation: https://picongpu.readthedocs.io/en/0.4.0/usage/workflows/tracerParticles.html

In [ ]:
# select time step
ts = 2500
# load calorimeter data with h5py
f_trace = h5.File(path_to_sim+"/simOutput/h5/simData_{}.h5".format(ts), "r")

for i in f_trace['/data/{}/particles/trace'.format(ts)].items():
    print(i)
    
print("----")

for i in f_trace['/data/{}/particles/trace/'.format(ts)].attrs.items():
    print(i)
f_trace.close()

In [ ]:
all_h5_ts = np.arange(100, 5000+1, 100)

time = np.zeros(len(all_h5_ts))
pos_x = np.zeros(len(all_h5_ts))
pos_y = np.zeros(len(all_h5_ts))


for i, ts in enumerate(all_h5_ts):
    f_trace = h5.File(path_to_sim+"/simOutput/h5/simData_{}.h5".format(ts), "r")
    time[i] = ts
    pos_x[i] = (f_trace['/data/{}/particles/trace/positionOffset/x'.format(ts)][:] +
                f_trace['/data/{}/particles/trace/position/x'.format(ts)][:])    
    pos_y[i] = (f_trace['/data/{}/particles/trace/positionOffset/y'.format(ts)][:] +
                f_trace['/data/{}/particles/trace/position/y'.format(ts)][:])

In [ ]:
plt.plot(time, pos_x, label="x")
plt.plot(time, pos_y, label="y")

plt.xlabel("t [time steps]", fontsize=18)
plt.xticks(fontsize=14)

plt.ylabel("position [cells]", fontsize=18)
plt.yticks(fontsize=14)

plt.legend(loc=2, fontsize=14)

plt.show()

In [ ]:
plt.plot(pos_y, pos_x)

plt.xlabel("y [cells]", fontsize=18)
plt.xticks(fontsize=14)

plt.ylabel("x [cells]", fontsize=18)
plt.yticks(fontsize=14)

plt.show()